In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!pip install python-telegram-bot==13.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached python_telegram_bot-13.8-py3-none-any.whl (495 kB)
  Attempting uninstall: python-telegram-bot
    Found existing installation: python-telegram-bot 13.13
    Uninstalling python-telegram-bot-13.13:
      Successfully uninstalled python-telegram-bot-13.13


In [25]:
!pip install python-telegram-bot --upgrade
!pip install dialogflow
!pip install google-cloud-dialogflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached python_telegram_bot-13.13-py3-none-any.whl (513 kB)
  Attempting uninstall: python-telegram-bot
    Found existing installation: python-telegram-bot 13.8
    Uninstalling python-telegram-bot-13.8:
      Successfully uninstalled python-telegram-bot-13.8


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
!pip install grpcio-status

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
!pip install google-api-core 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
!pip install google-cloud-vision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Урок 16.Курсовой проект. Создание чат-бота в Telegram

In [33]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from google.cloud import dialogflow

import numpy as np
import pandas as pd
import re

import fasttext

import pymorphy2

from stop_words import get_stop_words

from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [34]:
device = "cuda:0" 

### 1)Болталка(Dialogwlow)

In [35]:
updater = Updater("5512004151:AAECzq0qd2GMJZkGWRoWH9ymUSbcTE2Fgv4", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/drive/MyDrive/NLP_course_project/gb-couse-nlp-bot-lvse-bd743b726a0b.json'# скачaнный JSON


DIALOGFLOW_PROJECT_ID = 'gb-couse-nlp-bot-lvse' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'GB_couse_NLP_bot'  # ID бота из телеграмма

### 2) Рецепты

In [36]:
tokenizer_rec = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP_course_project/tokenizer_rec")
model_rec = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/NLP_course_project/model_rec")

In [37]:
def get_recipe(text):
  prefix = text
  tokens = tokenizer_rec(prefix, return_tensors='pt')

  size = tokens['input_ids'].shape[1]
  output = model_rec.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False,
    max_length=100, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
    length_penalty=0.5    
  )

  return tokenizer_rec.decode(output[0])

### 3)Медицинские ответы.


In [38]:
tokenizer_med = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP_course_project/tokenizer_med")
model_med = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/NLP_course_project/model_med")

In [39]:
def get_medicine(text):
  prefix = text
  tokens = tokenizer_med(prefix, return_tensors='pt')

  size = tokens['input_ids'].shape[1]
  output = model_med.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False,
    max_length=30, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
    length_penalty=1.5    
  )

  return tokenizer_med.decode(output[0])

### 4) Классификатор

In [40]:
classificator = fasttext.load_model("/content/drive/MyDrive/NLP_course_project/classificator.model")

### 5) Бот

In [41]:
stopwords = get_stop_words("ru")

morph = pymorphy2.MorphAnalyzer()

words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    text = str(text)
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]

def lemmatize(words, lemmer = morph, stopwords = stopwords):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):    
    return ' '.join((lemmatize(find_words(text))))

In [42]:
def detect_intent_texts(project_id, session_id, text, language_code):

    cleaned_text = preprocess(text)

    label = classificator.predict(cleaned_text)[0][0]
    
    if label == '__label__0':
      
      session_client = dialogflow.SessionsClient()
      session = session_client.session_path(project_id, session_id)
    
      text_input = dialogflow.TextInput(text=text, language_code=language_code)
      query_input = dialogflow.QueryInput(text=text_input)

      response = session_client.detect_intent(
          request={"session": session, "query_input": query_input}
      )

        
      return response.query_result.fulfillment_text

    elif label == '__label__1':

      text = get_recipe(text)
      return text

    else:
      text = get_medicine(text)
      return text    

In [43]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')

    
def textMessage(update: Update, context: CallbackContext):
    text_user = update.message.text
    
    
    text_bot = detect_intent_texts(DIALOGFLOW_PROJECT_ID, SESSION_ID, text_user, DIALOGFLOW_LANGUAGE_CODE)
    if text_bot:
        update.message.reply_text(text_bot)
    else:
        update.message.reply_text('Что?')

In [44]:
# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

In [45]:
# Start the Bot
updater.start_polling()
updater.idle()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
